In [1]:
import numpy as np
import pandas as pd
import missingno as msno
from datetime import datetime

In [2]:
## Load covid_cases and covid_fatalities
covid_cases = pd.read_csv('data/covid_cases.csv')
covid_fatalities = pd.read_csv('data/covid_fatalities.csv')
county_zip_map = pd.read_csv('data/zip_county_map.csv', index_col=0)

### Covid Cases

In [3]:
# second row is the column names and drop the information above
covid_cases.columns = covid_cases.iloc[1]
covid_cases = covid_cases.drop(index=[0,1])

# Last rows have no useful information
covid_cases = covid_cases.drop(index=[256,257,258,259,260,261])

In [4]:
# Reduce column names to dates
col_names = covid_cases.columns.to_list()
col_names[1:] = [name[6:] for name in col_names[1:]]
col_names[0] = 'county'
covid_cases.columns = col_names

In [5]:
# melt columns to rows
covid_cases = pd.melt(covid_cases, id_vars=['county'], 
             value_vars=covid_cases.columns[1:].to_list(), 
             var_name ='date', 
             value_name = 'caseCount',)
covid_cases = covid_cases.sort_values('county')

In [6]:
## Convert column to date time and add month and year column
covid_cases['date'] = pd.to_datetime(covid_cases['date'], format='%m-%d-%Y')
covid_cases['month'] = pd.DatetimeIndex(covid_cases['date']).month
covid_cases['year'] = pd.DatetimeIndex(covid_cases['date']).year

In [7]:
# Convert cases to int from object and get the mean casecount by month
covid_cases['caseCount'] = covid_cases['caseCount'].astype(int)
covid_cases_grouped = covid_cases.groupby(['year', 'month', 'county']).agg({'caseCount':'mean'})
covid_cases_grouped = covid_cases_grouped.sort_values(['year', 'month'])
covid_cases_grouped = covid_cases_grouped.reset_index()

### Covid Fatalities

In [8]:
# second row is the column names and drop the information above
covid_fatalities.columns = covid_fatalities.iloc[1]
covid_fatalities = covid_fatalities.drop(index=[0,1])


In [9]:
# Last row, 257, has no useful information
covid_fatalities = covid_fatalities.drop(index=257)

In [10]:
# Reduce column names to dates
col_names = covid_fatalities.columns.to_list()
col_names[1:] = [name[11:] for name in col_names[1:]]
col_names[0] = 'county'
covid_fatalities.columns = col_names

In [11]:
# melt columns to rows
covid_fatalities = pd.melt(covid_fatalities, id_vars=['county'], 
             value_vars=covid_fatalities.columns[1:].to_list(), 
             var_name ='date', 
             value_name = 'fatalities',)
covid_fatalities = covid_fatalities.sort_values('county')

In [12]:
## Convert column to date time and add month and year column
covid_fatalities['date'] = pd.to_datetime(covid_fatalities['date'], format='%m-%d-%Y')
covid_fatalities['month'] = pd.DatetimeIndex(covid_fatalities['date']).month
covid_fatalities['year'] = pd.DatetimeIndex(covid_fatalities['date']).year

In [13]:
# Convert cases to int from object and get the mean fatalities by month
covid_fatalities['fatalities'] = covid_fatalities['fatalities'].astype(int)
covid_fatalities_grouped = covid_fatalities.groupby(['year', 'month', 'county']).agg({'fatalities':'mean'})
covid_fatalities_grouped = covid_fatalities_grouped.sort_values(['county', 'year', 'month'])
covid_fatalities_grouped = covid_fatalities_grouped.reset_index()

### Compare Cases and Fatalities county names

In [14]:
# Convert Fatalities to lowercase
covid_fatalities_grouped['county'] = covid_fatalities_grouped['county'].str.lower()
covid_cases_grouped['county'] = covid_cases_grouped['county'].str.lower()

In [15]:
# Compare case_counties and fatality_counties
case_counties = covid_cases_grouped['county'].unique().tolist()
case_counties = [county.lower() for county in case_counties]
set(case_counties)

fatalities_counties = covid_fatalities_grouped['county'].unique().tolist()
fatalities_counties = [county.lower() for county in fatalities_counties]
set(fatalities_counties)

# fatalities_counties uses DE WITT instead of dewitt, and also has an unknown county
set(fatalities_counties) - set(case_counties)

{'de witt', 'unknown'}

In [16]:
## fatalities includes "de witt" instead of "dewitt".  Also includes "unknown"
covid_fatalities_grouped['county'].replace(to_replace='de witt', value='dewitt', inplace=True)
# covid_fatalities_grouped.drop()
covid_fatalities_grouped = covid_fatalities_grouped[covid_fatalities_grouped['county'] != 'unknown']

In [17]:
## Check that sets are the same. Should produce an empty set
fatalities_counties_grouped = covid_fatalities_grouped['county'].unique().tolist()
set(fatalities_counties_grouped) - set(case_counties)

set()

In [18]:
## Merge the two datasets
covid_merged = pd.merge(covid_fatalities_grouped, covid_cases_grouped, 
                        on =['year','month','county'])

print(f'fatalities shape: {covid_fatalities_grouped.shape}')
print(f'cases shape: {covid_cases_grouped.shape}')
print(f'merged shape: {covid_merged.shape}')

fatalities shape: (4826, 4)
cases shape: (4826, 4)
merged shape: (4826, 5)


### Attach zip codes to counties

In [19]:
# Convert all of the county names to lowercase
county_zip_map['county'] = county_zip_map['county'].str.lower()

In [20]:
# merge covid dataset to county zips to add zip codes to each county
covid_mapped = pd.merge(left=covid_merged, 
                             right = county_zip_map, 
                             how='left', 
                             left_on='county', 
                             right_on = 'county')
# covid_copy_mapped[covid_copy_mapped['county'] == 'el paso']

In [21]:
covid_mapped = covid_mapped.dropna(subset=['zip_code']).reset_index(drop=True)

### Padding dataset with 0s for all dates back to 01-01-2012

In [22]:
# Create datetime column, 'Time' in covid dataframe
covid_mapped['day'] = 1
covid_mapped['Time'] = pd.to_datetime(covid_mapped[['year','month','day']])
covid_mapped = covid_mapped.drop(['month', 'year', 'day'], axis=1)

In [23]:
# zips = county_zip_map['zip_code']


In [24]:
# Create list of dates from 01-01-2012 to most recent in covid data
dates_list = []
new_date = pd.to_datetime('01-01-2012')
end_date = covid_mapped['Time'].max()

while new_date != end_date:
    dates_list.append(new_date)
    new_date += pd.DateOffset(months=1)
dates = pd.DataFrame(dates_list)
dates.columns = ['Time']

In [25]:
# Add TX for merging and merge zips and time on state to create overall structure
county_zip_map['state'] = 'TX'
dates['state'] = 'TX'
dates = dates.merge(county_zip_map, how='left', on='state')

# Merge to original dataframe
covid_clean = dates.merge(covid_mapped, how='left', on=['Time', 'zip_code', 'county'])
covid_clean = covid_clean.fillna(0)
covid_clean = covid_clean.drop('state', axis=1)

In [27]:
covid_clean

,Time,zip_code,county,fatalities,caseCount
0,2012-01-01,75002.0,collin,0.000000,0.000000
1,2012-01-01,75010.0,dallas,0.000000,0.000000
2,2012-01-01,75013.0,collin,0.000000,0.000000
3,2012-01-01,75019.0,dallas,0.000000,0.000000
4,2012-01-01,75022.0,denton,0.000000,0.000000
...,...,...,...,...,...
24935,2021-08-01,78757.0,travis,1130.354839,96055.870968
24936,2021-08-01,78758.0,travis,1130.354839,96055.870968
24937,2021-08-01,78759.0,travis,1130.354839,96055.870968
24938,2021-08-01,79912.0,el paso,2781.225806,140034.000000


In [28]:
# # Save csv
# covid_clean.to_csv('covid_clean.csv')